In [26]:
# Import library-library
import os
import glob
from tqdm import tqdm
import numpy as np

# Data Preparation and Preprocessing
import pandas as pd
import re
from string import digits

# Word Embedding
import joblib
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

# Input and Expansion Query
import nltk
nltk.download('punkt')
import math
from textblob import TextBlob
from yake import KeywordExtractor
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
from textblob import TextBlob
NLTK_StopWords = stopwords.words('indonesian')

# for 'who' using NER
import spacy
from spacy import displacy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ryand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if (((t.lower() == "tempat") or (t.lower() == "waktu") or (t.lower() == "hari")) or (t not in NLTK_StopWords))]
    return T

In [27]:
kata="ta ezks ,[p;,wse[f, [IWHF 8pigsiakdcns['pa{dhfO7P89w btcas9e8finuasen}]]]] sekali"
print(preprocessing(kata))

['ta', 'ezks', 'p', 'wse', 'f', 'iwhf', '8pigsiakdcns', 'pa', 'dhfo7p89w', 'btcas9e8finuasen']


In [28]:
#test spacy 
# to load download by running this  "python -m spacy download en_core_web_sm " on terminal first
nlp = spacy.load('en_core_web_sm')
sentence = "Apple is looking at buying U.K. startup for $1 billion"
# contohkalimatngasal = "The Mars Orbiter Mission (MOM), informally known as Mangalyaan, was launched into Earth orbit on 5 November 2013 by the Indian Space Research Organisation (ISRO) and has entered Mars orbit on 24 September 2014. India thus became the first country to enter Mars orbit on its first attempt. It was completed at a record low cost of $74 million."
contohkalimatngasal = "TA kok susah bgt ya ajg kata adi perwira ryan danu saputra "
contohkalimatngasal = "dosen bernama bu maryamah"

# doc = nlp(sentence)
# doc = nlp(kata)
doc = nlp(contohkalimatngasal)
print ([token.text for token in doc])

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
  for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_) 

In [ ]:
spacy.explain("GPE") 

In [ ]:
displacy.render(doc,style="ent",jupyter=True)
displacy.render(doc,style="dep",jupyter=True)

In [4]:

df_test = pd.read_csv('df_test.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_test.info())
print ('-'*90)
document_text_test= joblib.load('document_text_test.pkl')
print(len(document_text_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 0 to 146
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        147 non-null    object
 1   date         147 non-null    object
 2   description  147 non-null    object
 3   source       147 non-null    object
dtypes: object(4)
memory usage: 5.7+ KB
None
------------------------------------------------------------------------------------------
147


In [5]:
df_train = pd.read_csv('df_train.csv')
df_train = df_train[pd.notnull(df_train['description'])]
print(df_train.info())
print ('-'*90)
document_text_train= joblib.load('document_text_train.pkl')
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1313
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1314 non-null   object
 1   date         1314 non-null   object
 2   description  1314 non-null   object
 3   source       1314 non-null   object
dtypes: object(4)
memory usage: 51.3+ KB
None
------------------------------------------------------------------------------------------
1314


In [6]:
def bow_read(bow):
    bow_read = pd.read_csv(bow)
    bow_text = []

    for i in range(0,bow_read.shape[0]):
        if(i not in bow_text and i!=0):
            bow_text.append(bow_read.iloc[i,1])
        
    return(bow_read,bow_text)

In [7]:
def cari_dokpertama(kueriAsli):
    kueriPre=preprocessing(kueriAsli)
    kueriPre= " ".join (kueriPre)
    hasilSearch=[]
    tfidf_train = joblib.load('tfidf_train.pkl')
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    query_vec= tfidf_vectorizer.transform([kueriPre])
    # print('queryvec')
    print(query_vec)
    results=cosine_similarity(tfidf_train, query_vec).reshape((-1))
    # print(results)
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_train.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    return hasilSearch



In [8]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch):
    # print(hasilSearch)
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=50)
    # print(k_extractor)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=50)
    # print(k_extractor2)
    keywords = k_extractor.extract_keywords(text=hasilSearch)
    # print("pertama : ",keywords)
    keywords = k_extractor2.extract_keywords(text=hasilSearch)
    # print("kedua : ",keywords)
    keywordYake = [x for x, y in keywords]
    # print(keywordYake)
    #keywordYake.append(keywords)
    # print (keywordYake)
    return keywordYake
#print("Keywords of article\n", keywords)

In [9]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch):

    keywordtfidf=[]
    keywordtfidf2=[]

    #doc = 'بَاب فرض الْوضُوء وسننه وهيآته وَفرض الْوضُوء سِتّ خِصَال النِّيَّة عمند غسل الْوَجْه وَغسل الْوَجْه وَغسل الذراعين مَعَ الْمرْفقين وَمسح مَا قل من الرَّأْس وَغسل الرجلَيْن مَعَ الْكَعْبَيْنِ وَالتَّرْتِيب وعَلى قَول الْوَلَاء وسننه عشر خِصَال خمس مِنْهَا قبل غسل الْوَجْه وَهِي التَّسْمِيَة وَغسل الْكَفَّيْنِ والمضمضة وَالِاسْتِنْشَاق وَالْمُبَالغَة فيههما إِلَّا للصَّائِم وَخمْس بعد غسل الْوَجْه وَهِي تَقْدِيم الْيُمْنَى على ليسرى وَمسح جَمِيع الرَّأْس وَمسح الْأُذُنَيْنِ ظاهرهما وباطنهما وَإِدْخَال الأصبعين فيهمَا وتخليل أَصَابِع الرجلَيْن . وَغسل دَاخل الْكَعْبَيْنِ وَلَيْسَ مسح لعنق من سنَنه وفضيلته تكراره ثَلَاثًا وزالواجب فِيهِ مرّة والمرتان أفضل وَالثَّلَاث أكمل وهيآته أَن يبْدَأ فِي تَطْهِير الْأَعْضَاء بمواضع الِابْتِدَاء . فَإِن اقْتصر على فروضه استة أَجزَأَهُ وَإِن ضيع حَظّ نَفسه فِيمَا ترك'
    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    #print (total_words)
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys())
        #print(list(result.keys()))        
        return hasil
    #print(get_top_n(tf_idf_score, 25))
    #print(len(get_top_n(tf_idf_score, 1)))
    keywordtfidf.append(get_top_n(tf_idf_score, 35))
    for i in range(len(keywordtfidf)):
        #print (i)
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            #print (j)
            keywordtfidf2.append(keywordtfidf[i][j])
    #print (keywordtfidf2)
    return keywordtfidf2

In [10]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch):
    # print(hasilSearch)

    keywordbert=[]

    #for j in range(len(array_text)):
    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 1))
    # print(keyword1)
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 2))
    # print(keyword2)
    # keyword3 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 5))
    # print(keyword3)

    #print("Keywords of article\n", keywords)
    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    # print (keywordbert)
    # print(len(keywordbert))
    return keywordbert

In [11]:
# hasilDokumenWhen=[]
# kueriAsliWhen='hari apa waktu terjadinya'
# hasilSearch=cari_dokpertama(kueriAsliWhen)
# keywordber=keyword_bert(hasilSearch)

In [12]:
def rangking (keywordGabung):
    keywordTemp=[]
    keywordFinal=[]

    def borda_sort(lists):
        scores = {}
        for l in lists:
            for idx, elem in enumerate(reversed(l)):
                if not elem in scores:
                    scores[elem] = 0
                scores[elem] += idx
        return sorted(scores.keys(), key=lambda elem: scores[elem], reverse=True)

    keywordTemp.append(borda_sort(keywordGabung))
    print ('kandidat temp',keywordTemp)
    print ('Total Kandidat temp: ',len(keywordTemp[0]))

    if len(keywordTemp[0])>30:
        print ('kurang dari 80')
        for i in range (0,80):
            keywordFinal.append(keywordTemp[0][i])
    elif len(keywordTemp[0])<80:
        print ('lebih dari 80')
        for i in range (0,len(keywordTemp)):
            for j in range (0,len(keywordTemp[0])):
                keywordFinal.append(keywordTemp[0][j])
    print ('Total Kandidat final: ',len(keywordFinal))
    print ('Kandidat final: ',keywordFinal)
    return keywordFinal


In [13]:
def keyword_BOW(keywordBOW, kueriAsli):
    cekDuplicate = []
    kandidatFix = []

    for i in keywordBOW:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    # queries=[kueriAsli]
    queries=kueriAsli
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    # closest_n = 1000
    closest_n = 2000
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    return kandidatFix

In [14]:
def kandidatFix(kueriAsli,bow):
    #kueri what
    # kueriAsli='apa sebenarnya kejadian yang terjadi diberita tersebut'
    kueri=preprocessing(kueriAsli)
    kueri= [" ".join (kueri)]
    print (kueri)
    hasilkandidat=[]
    keywordGabung=[]
    kandidatFix=[]
    kueriFix=[]
    kueriFixWithDelimiter=[]
    


    hasilSearch=cari_dokpertama(kueriAsli)
    keywordYake=keyword_yake(hasilSearch)
    keywordtfidf2=keyword_tfidf(hasilSearch)
    keywordbert=keyword_bert (hasilSearch)
    keywordBOW=keyword_BOW(bow, kueri)

    keywordGabung.append(keywordYake)
    keywordGabung.append(keywordtfidf2)
    keywordGabung.append(keywordbert)
    # keywordGabung.append(keywordBOW)
    hasilrank=rangking(keywordGabung)

    for i in hasilrank:
        kueriFix.append(i)
    for x in keywordBOW:
        kueriFix.append(x)
    for j in kueriFix:
        hasilkandidat.append(j)
    kueriFixWithDelimiter=kueriFix
    kueriFix=[preprocessing(i) for i in kueriFix]
    for i in kueriFix:
        for j in i:
            kandidatFix.append(j)
    kandidatFix= [" ".join (kandidatFix)]
    
    print ('*'*120)
    return(kandidatFix,keywordGabung,keywordBOW,hasilrank,kueriFixWithDelimiter)


In [15]:
bow_ekonomi_read,bow_ekonomi_text = bow_read('bow_ekonomi.csv')
bow_where_read,bow_where_text = bow_read('bow_where_negara.csv')
bow_when_read,bow_when_text = bow_read('bow_when.csv')
bow_who_read,bow_who_text = bow_read('bow_who_ekonomi.csv')

In [16]:
print("What     : ", bow_ekonomi_text[0])
print("Where    : ", bow_where_text[0])
print("When     : ", bow_when_text[0])
print("Who      : ", bow_who_text[0])

What     :  perekonomian
Where    :  surabaya
When     :  waktu
Who      :  produsen


In [17]:
bow_ekonomi_read.head()

,tingkat setelah parent,parent,similarity
0,1,ekonomi,"[('perekonomian', 0.7561678290367126), ('ekono..."
1,2,perekonomian,"[('ekonomi', 0.7561678290367126), ('ekonominya..."
2,2,ekonominya,"[('perekonomian', 0.676762044429779), ('ekonom..."
3,2,politik,"[('politiknya', 0.6882383227348328), ('sosial'..."
4,2,sosial,"[('sosialnya', 0.6959102153778076), ('politik'..."


In [18]:
bow_where_read.head()

,tingkat setelah parent,parent,similarity
0,NaN,NaN,NaN
1,1.0,surabaya,"[('semarang', 0.7880793213844299), ('malang', ..."
2,2.0,semarang,"[('surabaya', 0.7880793213844299), ('magelang'..."
3,2.0,malang,"[('surabaya', 0.724402129650116), ('kediri', 0..."
4,2.0,jakarta,"[('surabaya', 0.6788262724876404), ('bandung',..."


In [19]:
bow_when_read.head()

,tingkat setelah parent,parent,similarity
0,NaN,NaN,NaN
1,1.0,waktu,"[('waktunya', 0.6795573234558105), ('jam', 0.5..."
2,2.0,waktunya,"[('waktu', 0.6795573830604553), ('hidupnya', 0..."
3,2.0,jam,"[('menit', 0.7242034673690796), ('detik', 0.65..."
4,2.0,sementara,"[('sedangkan', 0.6645538806915283), ('saat', 0..."


In [20]:
bow_who_read.head()

,tingkat setelah parent,parent,similarity
0,NaN,NaN,NaN
1,1.0,produsen,"[('eksportir', 0.7249486446380615), ('pemasok'..."
2,2.0,eksportir,"[('importir', 0.7687522172927856), ('pengekspo..."
3,2.0,pemasok,"[('penyuplai', 0.7209192514419556), ('produsen..."
4,2.0,pabrikan,"[('produsen', 0.6965401768684387), ('opel', 0...."


In [21]:
print(len(bow_when_text))
cekDuplicate1 = []
kandidatFix2 = []

for i in bow_ekonomi_text:
    if(i not in cekDuplicate1 and i!=0):
        cekDuplicate1.append(i)
print(len(cekDuplicate1))

11111
1965


In [22]:
# hasilDokumenWhen=[]
hasilDokumenWho=[]
# hasilDokumenWhere=[]
# hasilDokumenWhat=[]

# kueriAsliWhat='apa yang terjadi diberita tersebut'
# kueriAsliWhere='di kota atau negara manakah berita ini '
kueriAsliWho='siapa pelaku kegiatan ekonomi dalam berita ini'
# kueriAsliWhen='hari apa waktu terjadinya'

# kandidatFixWhat,keywordGabungWhat,keywordBOW_What, hasilrankWhat, kueriFixWithDelimiter_What = kandidatFix(kueriAsliWhat, bow_ekonomi_text)
# kandidatFixWhere,keywordGabungWhere,keywordBOW_Where, hasilrankWhere,kueriFixWithDelimiter_Where = kandidatFix(kueriAsliWhere, bow_where_text)
kandidatFixWho,keywordGabungWho,keywordBOW_Who, hasilrankWho, kueriFixWithDelimiter_Who = kandidatFix(kueriAsliWho, bow_who_text)
# kandidatFixWhen,keywordGabungWhen,keywordBOW_When, hasilrankWhen, kueriFixWithDelimiter_When = kandidatFix(kueriAsliWhen, bow_when_text)


['pelaku kegiatan ekonomi berita']
  (0, 10854)	0.35915174993615934
  (0, 6847)	0.3762695174727039
  (0, 4573)	0.21105327500368934
  (0, 2066)	0.8275794740379139
kandidat temp [['youtube', 'twitter', 'qatar', 'twitter peron', 'kewirausahaan', 'otoritas qatar', 'youtube rangkuman', 'akun youtube', 'video twitter', 'kebutuhan', 'pembangunan', 'efektivitas', 'kewirausahaan pemilik', 'mencegah kematian', 'mengangkut', 'tayangan video', 'pengangkut', 'qatar mengirimkan', 'rangkuman', 'video berjudul', 'pusat', 'handphone indonesia', 'mengurangi', 'pembangunan ekonomi', 'kematian', 'indonesia triwulan', 'mengirimkan', 'kemdikbud pembelajaran', 'mencegah', 'qatar informasi', 'pembongkaran', 'kumulatif pertumbuhan', 'bali', 'mengalami rangkuman', 'kumulatif', 'memenuhi kebutuhan', 'menerapkan', 'bertujuan digunakam', 'coronavac', 'kegiatan mengangkut', 'pembelajaran', 'kebijakan bali', 'peron stasiun', 'bantuan', 'pelaksanaan', 'mengimpor', 'melibatkan', 'modal kewirausahaan', 'kemdikbud', 'be

In [23]:
# print("kueriFixWithDelimiter_When", kueriFixWithDelimiter_When)
# print("keywordBOW_When", keywordBOW_When)

print("kueriFixWithDelimiter_Who", kueriFixWithDelimiter_Who)
print("keywordBOW_Who", keywordBOW_Who)

# print("kueriFixWithDelimiter_Where", kueriFixWithDelimiter_Where)
# print("keywordBOW_Where", keywordBOW_Where)

# print("kueriFixWithDelimiter_What", kueriFixWithDelimiter_What)
# print("keywordBOW_What", keywordBOW_What)

kueriFixWithDelimiter_Who ['youtube', 'twitter', 'qatar', 'twitter peron', 'kewirausahaan', 'otoritas qatar', 'youtube rangkuman', 'akun youtube', 'video twitter', 'kebutuhan', 'pembangunan', 'efektivitas', 'kewirausahaan pemilik', 'mencegah kematian', 'mengangkut', 'tayangan video', 'pengangkut', 'qatar mengirimkan', 'rangkuman', 'video berjudul', 'pusat', 'handphone indonesia', 'mengurangi', 'pembangunan ekonomi', 'kematian', 'indonesia triwulan', 'mengirimkan', 'kemdikbud pembelajaran', 'mencegah', 'qatar informasi', 'pembongkaran', 'kumulatif pertumbuhan', 'bali', 'mengalami rangkuman', 'kumulatif', 'memenuhi kebutuhan', 'menerapkan', 'bertujuan digunakam', 'coronavac', 'kegiatan mengangkut', 'pembelajaran', 'kebijakan bali', 'peron stasiun', 'bantuan', 'pelaksanaan', 'mengimpor', 'melibatkan', 'modal kewirausahaan', 'kemdikbud', 'bertujuan membangun', 'persen mencegah', 'studi efektivitas', '2020 suhariyanto', 'ekonomi', 'pemerintah', 'kegiatan', 'menerapkan kebijakan', 'berita be

In [24]:
# j=1

testing_data = []

tfidf_vectorizer = joblib.load('vectorizer.pkl')

#What
# for i in range(0, len(document_text_test)-1):
    
#     hasilWhat=[]

#     teks=df_test.iloc[i,-2]
#     tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

#     query_vec_What= tfidf_vectorizer.transform(kandidatFixWhat)
#     results_what=cosine_similarity(tfidf_matrix, query_vec_What).reshape((-1))
#     hasilDokumenWhat.append(df_test.iloc[i,2])
#     for a in kueriFixWithDelimiter_What:
#         cariW = re.findall(a,hasilDokumenWhat[i])
#         #print(cariW)
#         if cariW:
#             hasilWhat.append(a)
    
#     data = [i,df_test.iloc[i,2],'what',kueriAsliWhat,keywordBOW_What , keywordGabungWhat,kandidatFixWhat, hasilWhat, results_what,' ',' ']

#     testing_data.append(data)

#Where
# for i in range(0, len(document_text_test)-1):

#     hasilWhere=[]
#     truePos = 0

#     teks=df_test.iloc[i,-2]
#     tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

#     query_vec_Where= tfidf_vectorizer.transform(kandidatFixWhere)
#     results_where=cosine_similarity(tfidf_matrix, query_vec_Where).reshape((-1))
#     hasilDokumenWhere.append(df_test.iloc[i,2])
#     for a in kueriFixWithDelimiter_Where:
#         cariW = re.findall(a,hasilDokumenWhere[i])
#         #print(cariW)
#         if cariW:
#             hasilWhere.append(a)
#     # print(hasilWhere)
#     for x in hasilWhere:
#         for y in keywordBOW_Where:
#             if(x == y):
#                 truePos=1
    
#     data = [i,df_test.iloc[i,2],'where',kueriAsliWhere,keywordBOW_Where , keywordGabungWhere, kandidatFixWhere, hasilWhere, results_where,truePos,' ']

#     testing_data.append(data)

#Who
for i in range(0, len(document_text_test)-1):

    hasilWho=[]

    teks=df_test.iloc[i,-2]
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

    query_vec_Who= tfidf_vectorizer.transform(kandidatFixWho)
    results_who=cosine_similarity(tfidf_matrix, query_vec_Who).reshape((-1))
    hasilDokumenWho.append(df_test.iloc[i,2])
    for a in kueriFixWithDelimiter_Who:
        cariW = re.findall(a,hasilDokumenWho[i])
        #print(cariW)
        if cariW:
            hasilWho.append(a)
    
    data = [i,df_test.iloc[i,2],'who',kueriAsliWho,keywordBOW_Who , keywordGabungWho, kandidatFixWho, hasilWho, results_who,' ',' ']

    testing_data.append(data)

#When
# for i in range(0, len(document_text_test)-1):

#     hasilWhen=[]

#     teks=df_test.iloc[i,-2]
#     tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

#     query_vec_When= tfidf_vectorizer.transform(kandidatFixWhen)
#     results_when=cosine_similarity(tfidf_matrix, query_vec_When).reshape((-1))
#     hasilDokumenWhen.append(df_test.iloc[i,2])
#     for a in kueriFixWithDelimiter_When:
#         cariW = re.findall(a,hasilDokumenWhen[i])
#         #print(cariW)
#         if cariW:
#             hasilWhen.append(a)
    
#     data = [i,df_test.iloc[i,2],'when',kueriAsliWhen,keywordBOW_When , keywordGabungWhen, kandidatFixWhen, hasilWhen, results_when,' ',' ']

#     testing_data.append(data)


In [25]:
#write to csv
writer = pd.DataFrame(testing_data, columns=['No Document','Description', 'W','Pertanyaan', 'Keyword BOW', 'Keyword Gabung','kandidat fix','hasilW', 'Kemiripan', 'True Positif', 'True Negative'])
writer.to_csv('QE_Stat_testing_who.csv', index=False, sep=',')